In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
import pickle

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
def data_prep_model_2():
    df = pd.read_csv(r"Credit.csv")
    df = df.drop(columns=["Unnamed: 0"])
    ## column transforming
    numeric_columns = df.select_dtypes(include =[np.number]).columns.tolist()
    cat_columns = df.select_dtypes(exclude=[np.number]).columns.tolist()
    
    new_df = df.drop(columns=["Rating", "Limit", "Age"])
    numeric_columns2 = new_df.select_dtypes(include =[np.number]).columns.tolist()
    
    input_data = []
    
    for i in new_df.columns:
        if i in numeric_columns2:
            if i=="Education":
                print(f"Podaj wartosć dla parametru NUMERYCZNEGO: {i}")
                ed = [x for x in range(5,21)]
                val = None
                while val not in ed:
                    try:
                        print("Dostępne wartosci (5-20)")
                        val = int(input("Podaj wartosć...: \n"))
                    except ValueError:
                        print("Podaj wartosc numeryczną!!")
                input_data.append(val) 
            elif i=="Income": 
                print(f"Podaj wartosć dla parametru NUMERYCZNEGO: {i}")
                val = None
                while type(val)!=type(1):
                    try:
                        val = int(input("Podaj wartosć... (np 10000 $/rok): \n"))
                    except ValueError:
                        print("Podaj wartosc numeryczną!!")
                input_data.append(val/1000)
            else:
                print(f"Podaj wartosć dla parametru NUMERYCZNEGO: {i}")
                val = None
                while type(val)!=type(1):
                    try:
                        val = int(input("Podaj wartosć... : \n"))
                    except ValueError:
                        print("Podaj wartosc numeryczną!!")
                input_data.append(val)
            
        elif i=="Gender":
            print(f"Podaj wartosć dla parametru JAKOSCIOWEGO: {i}")
            print("Kobieta (k) czy meżczyzna (m)?")
            gender = ['k', 'm']
            val = None
            while val not in gender:
                val = input("Podaj płeć (k/m...: \n")
            if val=="k":
                input_data.append("Female")
            elif val=="m":
                input_data.append(" Male")
        
        elif i =="Student":
            print(f"Podaj wartosć dla parametru JAKOSCIOWEGO: {i}")
            student = ['y', 'n']
            val = None
            while val not in student:
                print("Student: TAK(y) czy NIE (n)?")
                val = input("Odpowiedz na pytanie...: \n")
            if val=="y":
                input_data.append("Yes")
            elif val=="n":
                input_data.append("No")
                 
        elif i == "Married":
            print(f"Podaj wartosć dla parametru JAKOSCIOWEGO: {i}")
            maried = ['y', 'n']
            val = None
            while val not in maried:
                print("Zamężny: TAK(y) czy NIE (n)?")
                val = input("Odpowiedz na pytanie...: \n")
            if val=="y":
                input_data.append("Yes")
            elif val=="n":
                input_data.append("No")
                
        #{'African American', 'Asian', 'Caucasian'}    
        elif i=="Ethnicity":
            print(f"Podaj wartosć dla parametru JAKOSCIOWEGO: {i}")
            rasa = ["a","k","m"]
            val = None
            while val not in rasa:
                print("Przynależnosć etniczna: Azjata (a), Kaukaska (k), Afro Amerykańska (m)")
                val = input("Podaj odpowiednią literę...: \n")
            if val=="a":
                input_data.append("Asian")
            elif val=="k":
                input_data.append("Caucasian")
            elif val =="m":
                input_data.append("African American")
    return input_data      

In [7]:
def model_calc_raiting_with_balance(prediction):
    ## dataframe loading
    df = pd.read_csv(r"Credit.csv")
    df = df.drop(columns=["Unnamed: 0"])
    ## column transforming
    
    numeric_columns = df.select_dtypes(include =[np.number]).columns.tolist()
    cat_columns = df.select_dtypes(exclude=[np.number]).columns.tolist()
    
    new_df = df.drop(columns=["Rating", "Limit", "Age"])
    numeric_columns2 = new_df.select_dtypes(include =[np.number]).columns.tolist()
    
    ## creating specialized transformers
    class MyLabelBinarizer(BaseEstimator,TransformerMixin):
        def __init__(self, *args, **kwargs):
            self.encoder = OneHotEncoder(*args, **kwargs)
        def fit(self, x, y=0):
            self.encoder.fit(x)
            return self
        def transform(self, x, y=0):
            return self.encoder.transform(x)

    class DataframeSelector(BaseEstimator, TransformerMixin):
        def __init__(self, atrib_names):
            self.atribute_names = atrib_names
        def fit(self, X, y=None):
            return self
        def transform(self, X, y=0):
            return X[self.atribute_names].values
    

    num_pipeline = Pipeline([
        ('selector', DataframeSelector(numeric_columns2)),
        ('std_scaler', StandardScaler())
    ])
    #categorial pipeline
    cat_pipeline = Pipeline([
        ('selector', DataframeSelector(cat_columns)),
        ('onehot', MyLabelBinarizer())
    ])
    ##union piplines
    full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipline", cat_pipeline)
    ])


    X_data = new_df[numeric_columns2+cat_columns]
    X_data_2 = X_data.append({"Income": prediction[0], "Cards" : prediction[1], "Education" : prediction[2],
                            "Gender" : prediction[3], "Student" : prediction[4], "Married" : prediction[5], 
                            "Ethnicity" : prediction[6], "Balance" : prediction[7]}, ignore_index=True)
    
    #prediction_df = pd.DataFrame([np.array(prediction)], columns=new_df.columns)
    

    full_pipeline.fit(X_data_2[numeric_columns2+cat_columns])
    prediction_trans=full_pipeline.transform(X_data_2[numeric_columns2+cat_columns])
    #print(np.array(X_data_2)[-1])
    #print(prediction_trans.toarray()[-1])


    filename = 'model_xgb_tuned_with_balance.sav' 
    model_calc_balance = pickle.load(open(filename, 'rb'))
    pred = model_calc_balance.predict(prediction_trans)
    
    return pred[-1]

### Uzycie modelu

In [8]:
### Model ratinig (bez balance w X)
x = data_prep_model_2()
y= model_calc_raiting_with_balance(x)
y

Podaj wartosć dla parametru NUMERYCZNEGO: Income
Podaj wartosć... (np 10000 $/rok): 
10000
Podaj wartosć dla parametru NUMERYCZNEGO: Cards
Podaj wartosć... : 
55
Podaj wartosć dla parametru NUMERYCZNEGO: Education
Dostępne wartosci (5-20)
Podaj wartosć...: 
5
Podaj wartosć dla parametru JAKOSCIOWEGO: Gender
Kobieta (k) czy meżczyzna (m)?
Podaj płeć (k/m...: 
k
Podaj wartosć dla parametru JAKOSCIOWEGO: Student
Student: TAK(y) czy NIE (n)?
Odpowiedz na pytanie...: 
n
Podaj wartosć dla parametru JAKOSCIOWEGO: Married
Zamężny: TAK(y) czy NIE (n)?
Odpowiedz na pytanie...: 
n
Podaj wartosć dla parametru JAKOSCIOWEGO: Ethnicity
Przynależnosć etniczna: Azjata (a), Kaukaska (k), Afro Amerykańska (m)
Podaj odpowiednią literę...: 
a
Podaj wartosć dla parametru NUMERYCZNEGO: Balance
Podaj wartosć... : 
90
[10.0 55 5 90 'Female' 'No' 'No' 'Asian']
[-0.99803779 17.69351072 -2.67945458 -0.93428732  0.          1.
  1.          0.          1.          0.          0.          1.
  0.        ]
[18:00:42

224.60739